In [1]:
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests
from langchain_groq import ChatGroq
from dotenv import load_dotenv

load_dotenv()


True

In [4]:
@tool
def divide(a: int, b: int) -> float:
    """Takes two integers and returns the division result."""
    return a / b

In [6]:
print(divide({"b": 2, "a": 10}))

5.0


In [8]:
print(divide.name)
print(divide.description)
print(divide.args)

divide
Takes two integers and returns the division result.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


##### Tools Binding

In [9]:
llm = ChatGroq(
    model_name="llama3-8b-8192",
)

In [15]:
llm_with_tools = llm.bind_tools([divide])

In [17]:
llm_with_tools.invoke("Hi How are you?")

AIMessage(content='<tool-use>\n{\n  "tool_calls": []\n}\n</tool-use>', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 925, 'total_tokens': 942, 'completion_time': 0.014166667, 'prompt_time': 0.115190566, 'queue_time': 0.23593217100000002, 'total_time': 0.129357233}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-60053f40-93a3-40d4-880e-bd1d7e0c01ef-0', usage_metadata={'input_tokens': 925, 'output_tokens': 17, 'total_tokens': 942})

In [61]:
query = HumanMessage(content="Can you Divide 8 by 2.")

In [62]:
messages = [query]

In [63]:
messages

[HumanMessage(content='Can you Divide 8 by 2.', additional_kwargs={}, response_metadata={})]

In [64]:
result = llm_with_tools.invoke(messages)

# this is just suggestion from the llm execution part is later in this notebook


In [65]:
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_s4en', 'function': {'arguments': '{"a":8,"b":2}', 'name': 'divide'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 929, 'total_tokens': 1008, 'completion_time': 0.065833333, 'prompt_time': 0.11601687, 'queue_time': 0.021345731000000007, 'total_time': 0.181850203}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-301480e3-f580-4ec0-a005-3c2f957a8540-0', tool_calls=[{'name': 'divide', 'args': {'a': 8, 'b': 2}, 'id': 'call_s4en', 'type': 'tool_call'}], usage_metadata={'input_tokens': 929, 'output_tokens': 79, 'total_tokens': 1008})

In [66]:
messages.append(result)

In [67]:
messages

[HumanMessage(content='Can you Divide 8 by 2.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_s4en', 'function': {'arguments': '{"a":8,"b":2}', 'name': 'divide'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 929, 'total_tokens': 1008, 'completion_time': 0.065833333, 'prompt_time': 0.11601687, 'queue_time': 0.021345731000000007, 'total_time': 0.181850203}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-301480e3-f580-4ec0-a005-3c2f957a8540-0', tool_calls=[{'name': 'divide', 'args': {'a': 8, 'b': 2}, 'id': 'call_s4en', 'type': 'tool_call'}], usage_metadata={'input_tokens': 929, 'output_tokens': 79, 'total_tokens': 1008})]

In [68]:
result.tool_calls[0]['args']

{'a': 8, 'b': 2}

In [69]:
tool_result = divide.invoke(result.tool_calls[0])

# It returns the ToolMessage

In [70]:
messages.append(tool_result)

In [71]:
messages

[HumanMessage(content='Can you Divide 8 by 2.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_s4en', 'function': {'arguments': '{"a":8,"b":2}', 'name': 'divide'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 929, 'total_tokens': 1008, 'completion_time': 0.065833333, 'prompt_time': 0.11601687, 'queue_time': 0.021345731000000007, 'total_time': 0.181850203}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-301480e3-f580-4ec0-a005-3c2f957a8540-0', tool_calls=[{'name': 'divide', 'args': {'a': 8, 'b': 2}, 'id': 'call_s4en', 'type': 'tool_call'}], usage_metadata={'input_tokens': 929, 'output_tokens': 79, 'total_tokens': 1008}),
 ToolMessage(content='4.0', name='divide', tool_call_id='call_s4en')]

In [72]:
llm_with_tools.invoke(messages)

AIMessage(content='4.0', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 998, 'total_tokens': 1002, 'completion_time': 0.003333333, 'prompt_time': 0.124789862, 'queue_time': 0.02462198800000001, 'total_time': 0.128123195}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-fe0da4cc-a182-4fa8-85b9-a5e571973993-0', usage_metadata={'input_tokens': 998, 'output_tokens': 4, 'total_tokens': 1002})